In [82]:
import json
from tqdm import tqdm
# read json from data/json/tags.json and band.json
def remove_longest_string(lst):
    if not lst:
        return lst  # Return the list as is if it's empty
    
    # Find the longest string
    longest_string = max(lst, key=len)
    
    # Remove the longest string
    lst.remove(longest_string)
    
    return lst

def filter_keywords(keyword_list):
    filtered_keywords = []
    for keyword in keyword_list:
        if not keyword.isdigit():  # Check if the keyword is not only numbers
            if all(ord(char) < 128 for char in keyword):  # Check if keyword is English
                if len(keyword) >= 3:
                    filtered_keywords.append(keyword)
            else:
                filtered_keywords.append(keyword)
    return filtered_keywords


with open('data/json/tags.json', 'r') as f:
    tags = json.load(f)

tmp = []
for tag in tags:
    tmp.append(tag['name'])


tags = tmp

tmp = []
with open('data/json/band.json', 'r') as f:
    band = json.load(f)
    for b in band:
        band_keyword = remove_longest_string(b['band_keyword']) # 去除中英混合名字
        filtered_band_keyword = filter_keywords(band_keyword)
        tmp.extend(filtered_band_keyword)    
band = tmp
band

['蘭蔻',
 'lancome',
 'lancôme',
 '香奈兒',
 '迪奧',
 'dior',
 'whoo',
 'estée',
 'estéelauder',
 'estee lauder',
 '雅詩蘭黛',
 'estée lauder',
 'lauder',
 '契爾氏',
 "kiehl's",
 'kiehls',
 'loreal paris',
 '巴黎萊雅',
 "l'oreal paris",
 "l'orealparis",
 '霓淨思',
 'mac',
 '佳麗寶',
 '高絲',
 '資生堂',
 'clé',
 'cle de peau beaute',
 'clé de peau beauté',
 'clédepeaubeauté',
 'peau',
 '肌膚之鑰',
 '達爾膚',
 'dr.wu',
 'drwu',
 '肌研',
 'hadalabo',
 'skii',
 '克蘭詩',
 'clarins',
 'rocheposay',
 'la rocheposay',
 '理膚寶水',
 'larocheposay',
 '亞曼尼',
 'armani',
 'giorgio armani beauty',
 'giorgioarmanibeauty',
 'sofina',
 '蘇菲娜',
 '媚比琳',
 'maybelline',
 'mentholatum',
 '曼秀雷敦',
 '芭比波朗',
 'bobbi brown',
 'bobbi',
 'bobbibrown',
 '聖羅蘭',
 'ysl',
 'guerlain',
 '嬌蘭',
 '蘭芝',
 '植村秀',
 'shuuemura',
 'shu uemura',
 'shu',
 'uemura',
 '碧歐斯',
 'senka',
 'elizabetharden',
 'elizabeth',
 'arden',
 '伊麗莎白雅頓',
 'elizabeth arden',
 'origins',
 '品木宣言',
 "l'occitane",
 'loccitane',
 '歐舒丹',
 '雪花秀',
 'sulwhasoo',
 'clinique',
 '倩碧',
 'katetokyo',
 'kate 

In [83]:
# save tag as flat list
with open('data/json/tags_flat.json', 'w') as f:
    json.dump(tags, f, ensure_ascii=False, indent=4)    

In [84]:
len(tags)

774

In [85]:

# Step 1: Extraction

with open('data/json/makeup.json', 'r') as f:
    makeup = json.load(f)




In [86]:
import re
def merge_push_content_by_userid(messages):
    result = {}
    for message in messages:
        userid = message["push_userid"]
        content = message["push_content"]
        if userid in result:
            result[userid].append(content)
        else:
            result[userid] = [content]
    output = []
    for k,v in result.items():
        output.append(v)
    return output

def is_review_post(post_title):
    # Check if the title starts with "[心得]"
    return post_title.startswith("[心得]")

def find_elements_in_string(input_string, elements_list):
    # Convert the string to lower case to make the search case-insensitive
    input_string = input_string.lower()

    # Initialize an empty list to store the elements found in the string
    found_elements = []

    # Iterate through each element in the elements list
    for element in elements_list:
        # Check if the element is present in the string
        if element.lower() in input_string:
            # If present, add the element to the found_elements list
            found_elements.append(element)

    # Return the list of found elements
    return found_elements

def remove_punctuation(input_string):
    # Define a regular expression pattern to match all punctuation marks
    pattern = r'[^\w\s]'
    # Use re.sub to replace all punctuation marks with an empty string
    cleaned_string = re.sub(pattern, '', input_string)
    return cleaned_string

def remove_links(text):
    # Regular expression pattern to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute matched URLs with an empty string
    cleaned_text = re.sub(url_pattern, '', text)
    return cleaned_text
# Step 2: Transformation
posts = []


query_tag = "防曬"

for i,_ in enumerate(makeup):
    if query_tag not in _['content']:
        # print(f"Skip post {i} because it does not contain the query tag '{query_tag}'")
        continue
    if _['article_title'] and not is_review_post(_['article_title']) :
        continue
    post ={}
    post['title'] = _['article_title']
    post['tags'] = find_elements_in_string(_['content'], tags)
    post['band'] = find_elements_in_string(_['content'], band)
    post['content'] = remove_punctuation(remove_links(_['content']))
    messages = merge_push_content_by_userid(_['messages'])
    messages  = [remove_punctuation(remove_links(''.join(m))) for m in messages]
    post['messages'] = messages

    posts.append(post)

    


In [87]:
posts[1]

{'title': '[心得] 雅詩霧感唇膏#330-搶眼實搭的橘紅！',
 'tags': ['唇膏', '顯色', '霧面', '遮瑕', '防曬', '粉色系', '霧面唇膏'],
 'band': ['雅詩蘭黛'],
 'content': '最近一直被雅詩蘭黛絕對慾望霧感唇膏系列的代言照美到 尤其鋼鐵V擦430敬女人真的超有女神風範 被鋼鐵V詮釋得很棒 整個被形象照美的嫑嫑的 於是昨天下班就直接衝去櫃上試擦 btw我是去復興sogo靠櫃本來很害怕會被櫃姐冰畢竟下班後的狼狽樣真的很慘不忍 睹但還好我遇到蠻熱情而且很溫柔的櫃姐心情直接好到飛天 那麼就來分享一下擦照吧 先附上恐怖無比的素唇照 對暗沈唇跟唇紋有恐懼者 可以先按叉叉或用神之速度滑到最下面再慢慢滑上來看試色照   這沒有用任何對比或是調暗過啊 本人我的唇就是看起來這麼病容 好趕快跳過這裡 客官們可以回神了 在靠櫃之前我有先做好功課本來想考慮偏豆沙粉色系 但現場試的時候我真的會被我不聽話的素唇氣死 乍看還以為被我吃掉了 我還以為可以因此購入第一支豆沙色結果還是慘敗 後來請櫃姐挑一個比較適合我唇的顏色選了絲霧唇膏330 高調搶眼 櫃姐還說這支不會再被吃掉了忍不住笑出來 上圖  客官阿 連我不爭氣的素唇也這麼顯色 是不是太優秀了流淚 上後顏色我覺得很襯皮膚而且是不挑膚色的襯 算是正紅色但又不是很正我在說什麼 是一個會給人氣色好的印象而不是老娘是女王的那種紅 而且雖然是霧面唇膏 但不會乾 唇紋也不明顯 效果意外地好 最後上個半臉照   妝容就是防曬遮瑕眉毛跟隱眼而已上班真的連底都不想擦超懶 但由圖可知氣色整個就被唇膏給帶起來啦真的好愛你哦330 第一次發文希望大家鞭小力一點謝謝3 謝謝我素唇真的不ok 真的超級美我也打算週年慶去買哈哈哈 謝謝大家3',
 'messages': ['這反差有點厲害顏色好看',
  '這次的霧感唇膏真的超美週年慶想買個兩支',
  '顏色好美',
  '好看',
  '顏色適合妳',
  '美呆了',
  '顏色很適合你唇嘟嘟的擦得很好看',
  '好看',
  '好看耶被雅詩這一系列生火了XD覺得Vivian代言真的超級超級燒',
  '這顏色好美喔很遮妳的唇色我覺得這系列霧面的都不乾很好擦',
  '妳害我抑制半年多的唇膏慾又浮現了_',
  '顏色真的很美好看好看',
 

In [88]:
def split_string_fixed_size(input_string, size):
    """
    Splits the input string into a list of strings each of fixed size.

    Args:
    input_string (str): The string to be split.
    size (int): The fixed size for each split part.

    Returns:
    list: A list of strings split by the fixed size.
    """
    return [input_string[i:i + size] for i in range(0, len(input_string), size)]

    
posts_chunk = []
fixed_content_size = 200
for p in posts:
    if not p['title']:
        title = ''
    else:
        title = p['title']
    user_review = '使用者評價:'
    review = ''
    title = title.replace('[心得]','心得:')

    for i, message in enumerate(p['messages']):
        user_review += f"\n{i+1}. {''.join(message)}"
        
    band = ' '.join(p['band'])
    tags = ' '.join(p['tags'])
    
    band = '品牌:' + band + '\n'
    tags = '標籤:' + tags + '\n'
    
    # content = split_string_fixed_size(p['content'], fixed_content_size)
    # for c in content:
    #     posts_chunk.append(title+'\n'+c+'\n')    
    user_review = split_string_fixed_size(user_review, fixed_content_size)
    for u in user_review:
        posts_chunk.append(title+'\n'+u+'\n'+band+tags)

In [94]:
for i in range(20):
    print(posts_chunk[i])
    print()

心得: Dr.Althea 女神水光氣墊粉餅使用心得
使用者評價:
1. 眼睛好美喲
2. 剛完妝那張真是美翻了
3. 盒子好漂亮
4. 我覺得浮粉蠻嚴重的lol 不是會和出油融合的感覺
5. 近照感覺不太ok
6. 近照全臉都浮粉了吧 不過剛上很美
品牌:sofina 面膜 hello out you
標籤:液 粉底液 粉餅 蜜粉 遮瑕膏 粉撲 粉盒 面膜 韓國 台灣 遮瑕 自然 不悶 防曬 隔離 細紋 底妝 妝前 裸妝 眼下細紋 粉底 氣墊粉餅 敏感 補妝 控油 光澤感


心得: 雅詩霧感唇膏#330-搶眼實搭的橘紅！
使用者評價:
1. 這反差有點厲害顏色好看
2. 這次的霧感唇膏真的超美週年慶想買個兩支
3. 顏色好美
4. 好看
5. 顏色適合妳
6. 美呆了
7. 顏色很適合你唇嘟嘟的擦得很好看
8. 好看
9. 好看耶被雅詩這一系列生火了XD覺得Vivian代言真的超級超級燒
10. 這顏色好美喔很遮妳的唇色我覺得這系列霧面的都不乾很好擦
11. 妳害我抑制半年多的唇膏慾又浮現了_
12. 顏色真的很美
品牌:雅詩蘭黛
標籤:唇膏 顯色 霧面 遮瑕 防曬 粉色系 霧面唇膏


心得: 雅詩霧感唇膏#330-搶眼實搭的橘紅！
好看好看
13. 顏色好美
14. 原PO這顏色好好看耶 感覺秋冬就是要來一隻這種楓葉紅唇膏
15. 顏色太美了也想去靠櫃試試看
16. 這顏色很美很適合你
17. 顏色很漂亮擦起來很美
18. 好漂亮很適合喔
品牌:雅詩蘭黛
標籤:唇膏 顯色 霧面 遮瑕 防曬 粉色系 霧面唇膏


心得: 東京購入CANMAKE HAUL-多樣新品心得
使用者評價:
1. 好滿足
2. 我覺得貝殼腮紅霜滿雷的 顯色度很差
3. 看的超開心我也很喜歡Canmake
4. 我代購買的腮紅刷沒有刷具袋QQ
5. 有canmake推貝殼腮紅也推推擦起來蠻漂亮的
6. 那個眉粉個人也覺得很難用
7. 最近剛從沖繩回來 也買了PW39 覺得比PW38更自然 不易失手 Canmake 真的很好買
8. 推推
9. 我想看花花09ω
10. canmake腦粉給
品牌:mer 明色 eyes ascara arin aid conc juicy free 日本 base out gel add mai 東京 you beige
標籤:液 膠 睫毛膏 眉筆 眉粉

In [90]:
# Step 3: Load save posts_chunk to file
with open('data/json/posts_chunk_防曬.json', 'w') as f:
    json.dump(posts_chunk, f, ensure_ascii=False, indent=4)


In [91]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class PTTDocumentLoader(BaseLoader):
    def __init__(self, file_path: str) -> None:
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments

        with open(self.file_path, 'r') as f:
            makeup = json.load(f)

            line_number = 0
            for line in makeup:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

    # alazy_load is OPTIONAL. If you leave out the implementation, a default implementation which delegates to lazy_load will be used!
    async def alazy_load(self) -> AsyncIterator[Document]:  # <-- Does not take any arguments
        async with aiofiles.open(self.file_path, 'r', encoding='utf-8') as f:
            contents = await f.read()
            makeup = json.loads(contents)
    
            line_number = 0
            for line in makeup:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

In [92]:


loader = PTTDocumentLoader("data/json/posts_chunk.json")

In [93]:
## Test out the lazy load interface
for doc in loader.lazy_load():
    print()
    print(type(doc))
    print(doc)

FileNotFoundError: [Errno 2] No such file or directory: 'data/json/posts_chunk.json'